In [1]:
import sys
sys.path.append('/opt/splunk/etc/apps/multimodal-datagen/bin')

In [1]:
from clarifai_grpc.grpc.api import service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc
import traceback

ModuleNotFoundError: No module named 'clarifai_grpc'

In [2]:
def detectImage(imageURL, modelType, confidenceScore, addImageDescription):
    model = ''
    if modelType == 'NSFW':
        model = 'e9576d86d2004ed1a38ba0cf39ecb4b1'
    elif modelType == 'Apparel':
        model = '72c523807f93e18b431676fb9a58e6ad'
    elif modelType == 'Celebrity':
        model = 'cfbb105cb8f54907bb8d553d68d9fe20'
    elif modelType == 'Face':
        model = 'f76196b43bbd45c99b4f3cd8e8b40a8a'
    elif modelType == 'Logo':
        model = 'c443119bf2ed4da98487520d01a0b1e3'
    elif modelType == 'Color':
        model = 'eeed0b6733a644cea07cf4c60f87ebb7'
    allConcepts=''
    concepts = ''
    if addImageDescription =='True':
        if modelType== 'Demographics' or modelType== 'Travel' or modelType== 'Food' or modelType== 'General':
            try:
                stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())
                metadata = (('authorization', 'Key ed69d46b58a24e96aa7184ed2f635931'),)
                post_workflow_results_response = stub.PostWorkflowResults(
                    service_pb2.PostWorkflowResultsRequest(
                        workflow_id=modelType,
                        inputs=[
                            resources_pb2.Input(
                                data=resources_pb2.Data(
                                    image=resources_pb2.Image(
                                        url=imageURL
                                    )
                                )
                            )
                        ]
                    ),
                    metadata=metadata
                )
                if post_workflow_results_response.status.code != status_code_pb2.SUCCESS:
                    raise Exception("Post workflow results failed, status: " +
                                    post_workflow_results_response.status.description)
 
                results = post_workflow_results_response.results[0]
 
                if modelType== 'Demographics':
                    concepts = results.outputs[2].data.regions[0].data.concepts
                elif modelType== 'Travel' or modelType== 'Food' or modelType == 'General':
                    concepts = results.outputs[0].data.concepts
 
                for concept in concepts:
                    if concept.value > float(confidenceScore):
                        allConcepts=allConcepts+ ' '+concept.name
            except:
                f = open("/tmp/commands.txt", "w")
                f.write(str(traceback.format_exc()))
                f.close()
        else:
            try:
                channel = ClarifaiChannel.get_grpc_channel()
                stub = service_pb2_grpc.V2Stub(channel)
                metadata = (('authorization', 'Key ed69d46b58a24e96aa7184ed2f635931'),)
                post_model_outputs_response = stub.PostModelOutputs(
                    service_pb2.PostModelOutputsRequest(
                        model_id=model,
                        #version_id="{THE_MODEL_VERSION_ID}",  # This is optional. Defaults to the latest model version.
                        inputs=[
                            resources_pb2.Input(
                                data=resources_pb2.Data(
                                    image=resources_pb2.Image(
                                        url=imageURL
                                    )
                                )
                            )
                        ]
                    ),
                    metadata=metadata
                )
                if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
                    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)
                output = post_model_outputs_response.outputs[0]
                if modelType == 'Logo' or modelType== 'Apparel' or modelType== 'Face':
                    for region in output.data.regions:
                        for concept in region.data.concepts:
                            if concept.value > confidenceScore:
                                allConcepts=allConcepts+ ' '+concept.name
                elif modelType == 'Color':
                    concepts = output.data.colors
                elif modelType == 'Celebrity' or modelType== 'NSFW' :
                    concepts = output.data.concepts
                for concept in concepts:
                    if concept.value > float(confidenceScore):
                        if modelType == 'Color':
                            allConcepts=allConcepts+ ' '+concept.w3c. name
                        else:
                            allConcepts=allConcepts+ ' '+concept.name
            except:
                f = open("/tmp/commands.txt", "w")
                f.write(str(traceback.format_exc()))
                f.close()
    return allConcepts

In [5]:
test_img = 'http://35.246.69.64:8000/en-GB/static/app/Multimodal/videos/pHdxn4TSDLw/images/pHdxn4TSDLw-Scene-015-02.jpg'
detectImage(test_img, 'Logo', 0.3, 'True' )

' Huawei Jameson Whiskey'

In [ ]:
!pwd